In [3]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from disqco.circuits.QAOA import QAOA_random
from disqco.circuits.cp_fraction import cp_fraction
from disqco.parti.fgp.fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_CP_large_5it_2-4part_new.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_CP_large_5it_2-4part_new.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(128, 257, 32)

num_partitions_list = [2,4]

for num_partitions in num_partitions_list:
    for i, num_qubits in enumerate(sizes):
        # For each increase of 8 qubits, increase the number of partitions by 1


        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        
        # Sweep the fraction parameter from 0.1 to 0.9
            # Collect data for computing means across 10 iterations
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define/redefine circuit
            # -------------------------

            # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
            # Define the number of qubits and the circuit
            # circuit = QuantumVolume(num_qubits, depth=num_qubits)
            circuit = cp_fraction(num_qubits, num_qubits, fraction=0.5)
            # circuit = QFT(num_qubits, do_swaps=False)
            # circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
            # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
            basis_gates = ['cp', 'u']
            # basis_gates = ['cp','u']
            transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
            # Define the number of partitions
            # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
            # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
            # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.


            initial_partition = set_initial_partition_fgp(qpu_info=qpu_info,num_partitions=num_partitions)
            start = time.time()
            partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_info,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
            stop = time.time()

            total_time_fgp = stop - start
            min_cost_fgp = cost

            
            # -------------------------
            # 6. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "fgp_cost":  min_cost_fgp,
                "time_fgp": total_time_fgp,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            # Update detailed JSON right away
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        
        # ---------------------------------------------------------------------
        # After 10 iterations, compute the means and log them
        # ---------------------------------------------------------------------

        r_cost_list = [x["fgp_cost"] for x in iteration_data]
        

        r_time_list = [x["time_fgp"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        

        mean_r_time = float(np.mean(r_time_list))
        
        # Print to console for quick logging
        print("=============================================")
        print(f"Finished 10 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        # Store the aggregated means in a separate JSON
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,

            "mean_r_cost": mean_r_cost,

            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        # Update the means JSON file
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Initial cost of partition:  1998
Initial cost of partition:  2020
Initial cost of partition:  2014
Initial cost of partition:  2085
Initial cost of partition:  1991
Finished 10 iterations for:
  # Qubits: 128, # Partitions: 2
Mean Costs:
  Recursive:1418.800
Mean Times (s):
  Recursive:3.919
Initial cost of partition:  3196
Initial cost of partition:  3171
Initial cost of partition:  3252
Initial cost of partition:  3228
Initial cost of partition:  3211
Finished 10 iterations for:
  # Qubits: 160, # Partitions: 2
Mean Costs:
  Recursive:2184.400
Mean Times (s):
  Recursive:8.884
Initial cost of partition:  4631
Initial cost of partition:  4614
Initial cost of partition:  4626
Initial cost of partition:  4611
Initial cost of partition:  4594
Finished 10 iterations for:
  # Qubits: 192, # Partitions: 2
Mean Costs:
  Recursive:3149.600
Mean Times (s):
  Recursive:17.564
Initial cost of partition:  6312


KeyboardInterrupt: 

In [ ]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from QAOA import QAOA_random
from disqco.parti.fgp.fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_QV_5it_2-4part_new.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_QV_5it_2-4part_new.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

num_partitions_list = [2,4]

for num_partitions in num_partitions_list:
    for i, num_qubits in enumerate(sizes):
        # For each increase of 8 qubits, increase the number of partitions by 1


        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        
        # Sweep the fraction parameter from 0.1 to 0.9
            # Collect data for computing means across 10 iterations
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define/redefine circuit
            # -------------------------

            # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
            # Define the number of qubits and the circuit
            circuit = QuantumVolume(num_qubits, depth=num_qubits)
            # circuit = QFT(num_qubits, do_swaps=False)
            # circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
            # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
            basis_gates = ['cp', 'u']
            # basis_gates = ['cp','u']
            transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
            # Define the number of partitions
            # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
            # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
            # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.



            initial_partition = set_initial_partition_fgp(qpu_info=qpu_info,num_partitions=num_partitions)
            start = time.time()
            partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_info,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
            stop = time.time()

            total_time_fgp = stop - start
            min_cost_fgp = cost

            
            # -------------------------
            # 6. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "fgp_cost":  min_cost_fgp,
                "time_fgp": total_time_fgp,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            # Update detailed JSON right away
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        
        # ---------------------------------------------------------------------
        # After 10 iterations, compute the means and log them
        # ---------------------------------------------------------------------

        r_cost_list = [x["fgp_cost"] for x in iteration_data]
        

        r_time_list = [x["time_fgp"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        

        mean_r_time = float(np.mean(r_time_list))
        
        # Print to console for quick logging
        print("=============================================")
        print(f"Finished 10 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        # Store the aggregated means in a separate JSON
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,

            "mean_r_cost": mean_r_cost,

            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        # Update the means JSON file
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Initial cost of partition:  156
Initial cost of partition:  234
Initial cost of partition:  192
Initial cost of partition:  198
Initial cost of partition:  216
Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2
Mean Costs:
  Recursive:45.200
Mean Times (s):
  Recursive:0.017
Initial cost of partition:  438
Initial cost of partition:  438
Initial cost of partition:  444
Initial cost of partition:  450
Initial cost of partition:  474
Finished 10 iterations for:
  # Qubits: 24, # Partitions: 2
Mean Costs:
  Recursive:102.000
Mean Times (s):
  Recursive:0.053
Initial cost of partition:  822
Initial cost of partition:  792
Initial cost of partition:  750
Initial cost of partition:  810
Initial cost of partition:  822
Finished 10 iterations for:
  # Qubits: 32, # Partitions: 2
Mean Costs:
  Recursive:182.400
Mean Times (s):
  Recursive:0.155
Initial cost of partition:  1212
Initial cost of partition:  1242
Initial cost of partition:  1170
Initial cost of partition:  1188
Initial cos

In [1]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from QAOA import QAOA_random
from disqco.parti.fgp.fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_QV_5it_2-4part_new.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_QV_5it_2-4part_new.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

num_partitions_list = [4]

for num_partitions in num_partitions_list:
    for i, num_qubits in enumerate(sizes):
        # For each increase of 8 qubits, increase the number of partitions by 1


        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        

        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define/redefine circuit
            # -------------------------

            # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
            # Define the number of qubits and the circuit
            circuit = QuantumVolume(num_qubits, depth=num_qubits)
            # circuit = QFT(num_qubits, do_swaps=False)
            # circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
            # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
            basis_gates = ['cp', 'u']
            # basis_gates = ['cp','u']
            transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
            # Define the number of partitions
            # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
            # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 

            initial_partition = set_initial_partition_fgp(qpu_info=qpu_info,num_partitions=num_partitions)
            start = time.time()
            partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_info,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
            stop = time.time()

            total_time_fgp = stop - start
            min_cost_fgp = cost

            
            # -------------------------
            # 6. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "fgp_cost":  min_cost_fgp,
                "time_fgp": total_time_fgp,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            # Update detailed JSON right away
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        
        # ---------------------------------------------------------------------
        # After 10 iterations, compute the means and log them
        # ---------------------------------------------------------------------

        r_cost_list = [x["fgp_cost"] for x in iteration_data]
        

        r_time_list = [x["time_fgp"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        

        mean_r_time = float(np.mean(r_time_list))
        
        # Print to console for quick logging
        print("=============================================")
        print(f"Finished 10 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        # Store the aggregated means in a separate JSON
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,

            "mean_r_cost": mean_r_cost,

            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        # Update the means JSON file
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Initial cost of partition:  279
Initial cost of partition:  297
Initial cost of partition:  300
Initial cost of partition:  309
Initial cost of partition:  288
Finished 10 iterations for:
  # Qubits: 16, # Partitions: 4
Mean Costs:
  Recursive:89.800
Mean Times (s):
  Recursive:0.032
Initial cost of partition:  660
Initial cost of partition:  663
Initial cost of partition:  639
Initial cost of partition:  663
Initial cost of partition:  636
Finished 10 iterations for:
  # Qubits: 24, # Partitions: 4
Mean Costs:
  Recursive:198.200
Mean Times (s):
  Recursive:0.113
Initial cost of partition:  1194
Initial cost of partition:  1161
Initial cost of partition:  1119
Initial cost of partition:  1185
Initial cost of partition:  1152
Finished 10 iterations for:
  # Qubits: 32, # Partitions: 4
Mean Costs:
  Recursive:348.400
Mean Times (s):
  Recursive:0.267
Initial cost of partition:  1824
Initial cost of partition:  1875
Initial cost of partition:  1809
Initial cost of partition:  1782
Initia

In [1]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from QAOA import QAOA_random
from disqco.parti.fgp.fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_QFT_5it_2-4part_new.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_QFT_5it_2-4part_new.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

num_partitions_list = [2,4]

for num_partitions in num_partitions_list:
    for i, num_qubits in enumerate(sizes):
        # For each increase of 8 qubits, increase the number of partitions by 1


        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        
        # Sweep the fraction parameter from 0.1 to 0.9
            # Collect data for computing means across 10 iterations
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define/redefine circuit
            # -------------------------

            # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
            # Define the number of qubits and the circuit
            # circuit = QuantumVolume(num_qubits, depth=num_qubits)
            circuit = QFT(num_qubits, do_swaps=False)
            # circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
            # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
            basis_gates = ['cp', 'u']
            # basis_gates = ['cp','u']
            transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
            # Define the number of partitions
            # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
            # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
            # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.


            initial_partition = set_initial_partition_fgp(qpu_info=qpu_info,num_partitions=num_partitions)
            start = time.time()
            partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_info,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
            stop = time.time()

            total_time_fgp = stop - start
            min_cost_fgp = cost

            
            # -------------------------
            # 6. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "fgp_cost":  min_cost_fgp,
                "time_fgp": total_time_fgp,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            # Update detailed JSON right away
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        
        # ---------------------------------------------------------------------
        # After 10 iterations, compute the means and log them
        # ---------------------------------------------------------------------

        r_cost_list = [x["fgp_cost"] for x in iteration_data]
        

        r_time_list = [x["time_fgp"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        

        mean_r_time = float(np.mean(r_time_list))
        
        # Print to console for quick logging
        print("=============================================")
        print(f"Finished 10 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        # Store the aggregated means in a separate JSON
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,

            "mean_r_cost": mean_r_cost,

            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        # Update the means JSON file
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Initial cost of partition:  63
Initial cost of partition:  63
Initial cost of partition:  63
Initial cost of partition:  63
Initial cost of partition:  63
Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2
Mean Costs:
  Recursive:26.000
Mean Times (s):
  Recursive:0.007
Initial cost of partition:  143
Initial cost of partition:  143
Initial cost of partition:  143
Initial cost of partition:  143
Initial cost of partition:  143
Finished 10 iterations for:
  # Qubits: 24, # Partitions: 2
Mean Costs:
  Recursive:38.000
Mean Times (s):
  Recursive:0.019
Initial cost of partition:  255
Initial cost of partition:  255
Initial cost of partition:  255
Initial cost of partition:  255
Initial cost of partition:  255
Finished 10 iterations for:
  # Qubits: 32, # Partitions: 2
Mean Costs:
  Recursive:50.000
Mean Times (s):
  Recursive:0.047
Initial cost of partition:  399
Initial cost of partition:  399
Initial cost of partition:  399
Initial cost of partition:  399
Initial cost of partit

In [1]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from disqco.circuits.QAOA import QAOA_random
from disqco.circuits.cp_fraction import cp_fraction
from disqco.parti.fgp.fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time


###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_CP_2-12_new.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_CP_2-12_new.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

fraction_list = [0.3, 0.5, 0.7, 0.9]
sizes = range(16, 97, 8)

for fraction in fraction_list:
    for i, num_qubits in enumerate(sizes):
        # For each increase of 8 qubits, increase the number of partitions by 1
        num_partitions = 2 + i

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        
        # Sweep the fraction parameter from 0.1 to 0.9
            # Collect data for computing means across 10 iterations
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define/redefine circuit
            # -------------------------

            # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
            # Define the number of qubits and the circuit
            # circuit = QuantumVolume(num_qubits, depth=num_qubits)
            circuit = cp_fraction(num_qubits, num_qubits, fraction=fraction)
            # circuit = QFT(num_qubits, do_swaps=False)
            # circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
            # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
            basis_gates = ['cp', 'u']
            # basis_gates = ['cp','u']
            transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
            # Define the number of partitions
            # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
            # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
            # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.

            initial_partition = set_initial_partition_fgp(qpu_info=qpu_info,num_partitions=num_partitions)
            start = time.time()
            partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_info,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
            stop = time.time()

            total_time_fgp = stop - start
            min_cost_fgp = cost

            
            # -------------------------
            # 6. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "fraction" : fraction,
                "iteration": iteration,
                "fgp_cost":  min_cost_fgp,
                "time_fgp": total_time_fgp,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            # Update detailed JSON right away
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        
        # ---------------------------------------------------------------------
        # After 10 iterations, compute the means and log them
        # ---------------------------------------------------------------------

        r_cost_list = [x["fgp_cost"] for x in iteration_data]
        

        r_time_list = [x["time_fgp"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        

        mean_r_time = float(np.mean(r_time_list))
        
        # Print to console for quick logging
        print("=============================================")
        print(f"Finished 10 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions} , Fraction: {fraction}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        # Store the aggregated means in a separate JSON
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "fraction" : fraction,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        # Update the means JSON file
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Initial cost of partition:  19
Initial cost of partition:  14
Initial cost of partition:  19
Initial cost of partition:  15
Initial cost of partition:  26
Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2 , Fraction: 0.3
Mean Costs:
  Recursive:14.400
Mean Times (s):
  Recursive:0.003
Initial cost of partition:  48
Initial cost of partition:  63
Initial cost of partition:  55
Initial cost of partition:  50
Initial cost of partition:  53
Finished 10 iterations for:
  # Qubits: 24, # Partitions: 3 , Fraction: 0.3
Mean Costs:
  Recursive:49.400
Mean Times (s):
  Recursive:0.014
Initial cost of partition:  119
Initial cost of partition:  105
Initial cost of partition:  114
Initial cost of partition:  105
Initial cost of partition:  118
Finished 10 iterations for:
  # Qubits: 32, # Partitions: 4 , Fraction: 0.3
Mean Costs:
  Recursive:104.800
Mean Times (s):
  Recursive:0.042
Initial cost of partition:  197
Initial cost of partition:  192
Initial cost of partition:  185
Initial co

In [1]:
from QASMBench.interface.qiskit import QASMBenchmark

# path to the root directory of QASMBench
path = "QASMBench"
# selected category for QASMBench
category = "large" 
# select only the circuits with the number of qubits in the list
num_qubits_list = list(range(3, 100))
# whether to remove the final measurement in the circuit
remove_final_measurements = True
# whether to use qiskit.transpile() to transpile the circuits (note: must provide qiskit backend)
do_transpile = True  # Changed to True for transpiling circuits
# arguments for qiskit.transpile(). backend should be provided at least
transpile_args = { 'basis_gates': ['u', 'cp']}
bm = QASMBenchmark(path, category, num_qubits_list=num_qubits_list, do_transpile=do_transpile, **transpile_args, remove_final_measurements=remove_final_measurements)

circ_name_list = bm.circ_name_list

In [4]:
import os
import json
import numpy as np
from qiskit import transpile
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from QAOA import QAOA_random
from cp_fraction import cp_fraction
from disqco.parti.fgp.fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_QASM.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################

means_filename = "benchmark_means_FGP_QASM.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

num_partition_list = [2,3,4]


for num_partitions in num_partition_list:

    for i, circ_name in enumerate(circ_name_list):
        circ = bm.get(circ_name)
        num_qubits = circ.num_qubits
        circuit = circ
        circuit = transpile(circuit, basis_gates = ['u', 'cp'])
        if circuit.depth() < 1000:
            print(f"Starting benchmark for {num_qubits} qubits, {num_partitions} partitions, circuit: {circ_name_list[i]}")

            qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]

            
            iteration_data = []
            for iteration in range(5):

                initial_partition = set_initial_partition_fgp(qpu_info=qpu_info,num_partitions=num_partitions)
                start = time.time()
                partition, cost, mapping = main_algorithm(circuit=circuit, qpu_info=qpu_info,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
                stop = time.time()

                total_time_fgp = stop - start
                min_cost_fgp = cost


                result_entry = {
                    "circuit_name": circ_name_list[i],
                    "num_qubits": num_qubits,
                    "num_partitions": num_partitions,
                    "iteration": iteration,
                    "fgp_cost":  min_cost_fgp,
                    "time_fgp": total_time_fgp,
                }
                
                detailed_results.append(result_entry)
                iteration_data.append(result_entry)
                
                # Update detailed JSON right away
                with open(detailed_filename, "w") as f:
                    json.dump(detailed_results, f, indent=2)
            
            # ---------------------------------------------------------------------
            # After 5 iterations, compute the means and log them
            # ---------------------------------------------------------------------

            fgp_cost_list = [x["fgp_cost"] for x in iteration_data]
            fgp_time_list = [x["time_fgp"] for x in iteration_data]
            

            mean_fgp_cost = float(np.mean(fgp_cost_list))
            mean_fgp_time = float(np.mean(fgp_time_list))
            
            print("=============================================")
            print(f"Finished 10 iterations for:")
            print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}, circuit: {circ_name_list[i]}")
            print("Mean Costs:")

            print(f"  FGP:{mean_fgp_cost:.3f}")
            print("Mean Times (s):")

            print(f"  FGP:{mean_fgp_time:.3f}")
            print("=============================================")
            
            # Store the aggregated means in a separate JSON
            mean_entry = {
                "circuit_name": circ_name_list[i],
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "mean_fgp_cost": mean_fgp_cost,
                "mean_fgp_time": mean_fgp_time,
            }
            
            mean_results.append(mean_entry)
            
            # Update the means JSON file
            with open(means_filename, "w") as f:
                json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Starting benchmark for 28 qubits, 2 partitions, circuit: adder_n28
Initial cost of partition:  106
Initial cost of partition:  106
Initial cost of partition:  106
Initial cost of partition:  106
Initial cost of partition:  106
Finished 10 iterations for:
  # Qubits: 28, # Partitions: 2, circuit: adder_n28
Mean Costs:
  FGP:20.000
Mean Times (s):
  FGP:0.022
Starting benchmark for 29 qubits, 2 partitions, circuit: qft_n29
Initial cost of partition:  420
Initial cost of partition:  420
Initial cost of partition:  420
Initial cost of partition:  420
Initial cost of partition:  420
Finished 10 iterations for:
  # Qubits: 29, # Partitions: 2, circuit: qft_n29
Mean Costs:
  FGP:46.000
Mean Times (s):
  FGP:0.096
Starting benchmark for 30 qubits, 2 partitions, circuit: bv_n30


ValueError: too many values to unpack (expected 2)